<h1>Physical Distance maintaing among the crowd

<h2>Deep Learning<br>
<h5>Deep learning is a type of machine learning and artificial intelligence (AI) that imitates the way humans gain certain types of knowledge.<br>
 Deep learning is an important element of data science, which includes statistics and predictive modelling. It is extremely beneficial to data scientists who are tasked with collecting, analysing and interpreting large amounts of data deep learning makes this process faster and easier

<h4>Importing python modules

In [1]:
from scipy.spatial import distance as dist
import numpy as np
import imutils
import cv2
import os

<h4> Function for people detection

In [2]:
def detect_people(frame, net, ln, recPers=0):
	(H, W) = frame.shape[:2]
	results = []

	blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416),swapRB=True, crop=False) # scaling the frames to convert all the image
																						#in the range of 0-255
	net.setInput(blob)
	layerOutputs = net.forward(ln) #provide nparray to draw bounding box

	boxes = []  #bounding boxes
	centroids = [] #centeroids
	confidences = [] #probability list for detecting a  class 

	for output in layerOutputs:
		for detection in output:
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]

			#filtering persons from image and setting its probability
			if classID == recPers and confidence > 0.4:
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")

				#finding the corners
				x = int(centerX - (width / 2)) 
				y = int(centerY - (height / 2))

				boxes.append([x, y, int(width), int(height)])
				centroids.append((centerX, centerY))
				confidences.append(float(confidence))

	idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.2) #used for suppression overlapping bounding boxes

	if len(idxs) > 0:
		for i in idxs.flatten():
			# extract the bounding box coordinates
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])
			r = (confidences[i], (x, y, x + w, y + h), centroids[i])
			results.append(r)

	return results #returning the list consist of max person probability 
					#and bounding box co- ordinates


<h4>Importing the pretrained model

In [3]:
MODEL_PATH = "yolo-coco"

labelsPath = os.path.sep.join([MODEL_PATH, "coco.names"]) #list of names on which models were prepared
LABELS = open(labelsPath).read().strip().split("\n")

weightsPath = os.path.sep.join([MODEL_PATH, "yolov4-tiny.weights"]) 
configPath = os.path.sep.join([MODEL_PATH, "yolov4-tiny.cfg"])

#loading module into the program
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

In [4]:
ln = net.getLayerNames() #gives you a list of all layers used 
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()] #gives you the index of the layer

In [5]:
print("[INFO] accessing video stream...")
#cap = cv2.VideoCapture("pedestrians.mp4")
#cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture("vid_short.mp4")
cap = cv2.VideoCapture("input2.mp4")
writer = None

while True:
	# read the next frame from the file
	(ret, frame) = cap.read()

	if not ret:
		break

	frame = imutils.resize(frame, width=900)
	results = detect_people(frame, net, ln, recPers=LABELS.index("person"))

	violate = set() #it contains set of violation pair

	if len(results) >= 2:
		centroids = np.array([r[2] for r in results])
		D = dist.cdist(centroids, centroids, metric="euclidean")
		for i in range(0, D.shape[0]):
			for j in range(i + 1, D.shape[1]):
				if D[i, j] < 75.0:
					violate.add(i)
					violate.add(j)

	# We use BGR format for coloring, here we are coloring the bounding boxes
	for (i, (prob, bbox, centroid)) in enumerate(results):
		(startX, startY, endX, endY) = bbox
		color = (0, 255, 0)	

		#update color to red for violation
		if i in violate:
			color = (0, 0, 255)

		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

	#Output Frame
	text = "Vioations : {}".format(len(violate))
	cv2.putText(frame, text, (10, frame.shape[0] - 25),cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 0, 5), 3)

	#Displaying Results
	cv2.imshow("Frame", frame)
	if cv2.waitKey(1) & 0xFF==ord("q"):
		break

	if  writer is None:
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")

	if writer is not None:
		writer.write(frame)

cap.release() #releasing the resource

cv2.destroyAllWindows()	

[INFO] accessing video stream...
